### 환경 확인 및 이전에 생성한 train_data, test_data 테이블 재로딩.

In [0]:
spark.conf.get("spark.driver.memory")

In [0]:
%sh
vmstat 

In [0]:
%fs
ls dbfs:/user/hive/warehouse/train_data

In [0]:
%sql
drop table if exists train_data;

CREATE TABLE train_data
USING parquet
OPTIONS (
       path "/user/hive/warehouse/train_data/" );

drop table if exists test_data;

CREATE TABLE test_data
USING parquet
OPTIONS (
       path "/user/hive/warehouse/test_data/" );    

In [0]:
spark.catalog.listTables()

In [0]:
# create table using parquet 로 로딩한 테이블에 대해서 delta format 확인 하지 말것. 
spark.sql("set spark.databricks.delta.formatCheck.enabled=false")

### Table을 DataFrame으로 변경하고 전처리/벡터화 적용

In [0]:
# train_data, test_data 테이블을 DataFrame으로 변환.
train_sdf = spark.table("train_data") # spark.sql("select * from train_data")
test_sdf = spark.table("test_data") # spark.sql("select * from test_data")
print('train_sdf type:', type(train_sdf), 'test_sdf type:', type(test_sdf))
train_sdf.printSchema()

In [0]:
# Null값은 모두 0으로 설정. 
train_sdf = train_sdf.fillna(0)

In [0]:
from pyspark.ml.feature import VectorAssembler

# feature vectorization 적용할 column명 추출. label 컬럼인 reordered는 제외
vector_columns = [column_name for column_name, column_type in train_sdf.dtypes if column_name != 'reordered']

vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
train_sdf_vectorized = vector_assembler.transform(train_sdf)

display(train_sdf_vectorized.limit(10))

In [0]:
train_sdf_cnt = train_sdf.count()
print(train_sdf_cnt)

### LightGBM, (XGBoost)로 학습 및 예측
* XGBoost의 경우 databricks 커뮤니티 에디션의 메모리 부족으로 학습시 오류 발생. driver memory를 2G로 설정할 경우 수행 완료 (가끔) 가능.

In [0]:
from mmlspark.lightgbm import LightGBMClassifier
import mlflow 

lgbm_estimator = LightGBMClassifier(featuresCol="features", labelCol="reordered"
                                    , numLeaves=100, maxDepth=10, numIterations=100
                                    , earlyStoppingRound=15)

#mlflow.autolog(disable=True)
lgbm_model = lgbm_estimator.fit(train_sdf_vectorized)

In [0]:
%sh
vmstat

In [0]:
from sparkdl.xgboost import XgboostClassifier

# num_workers는 현재 memory 용량 제약으로 반드시 1
xgb_estimator = XgboostClassifier(num_workers=1, featuresCol="features", labelCol="label", max_depth=10)
xgb_model = xgb_estimator.fit(train_sdf_vectorized.limit(100000)) # 전체 데이터로 수행 시 driver memory를 2G로 설정해도 오류가 나는 경우가 있음. 

In [0]:
# submit 파일을 만들기 위한 order_id, product_id 용 test_sdf_id와 id가 제거된 feature 속성만을 가진 test_sdf 생성. 
def make_test_sdf_id_n_test_sdf(test_sdf):
    test_sdf_id = test_sdf.select('user_id', 'product_id', 'order_id')
    test_sdf = test_sdf.drop('user_id', 'product_id', 'order_id')
    return test_sdf_id, test_sdf

test_sdf_id, test_sdf = make_test_sdf_id_n_test_sdf(test_sdf)

In [0]:
# test 데이터 세트로 예측하고 결과를 predictions DataFrame으로 생성. 
test_sdf = test_sdf.fillna(0)
test_sdf_vectorized = vector_assembler.transform(test_sdf)
predictions = lgbm_model.transform(test_sdf_vectorized)
display(predictions.limit(10))

### Prediction결과 probability의 threshold는 0.21로 재조정하여 재 예측하고 submit용 결과 파일로 재 생성.

In [0]:
#예측 결과 predictions와 test_sdf_id를 조인하여 붙이기.
from pyspark.sql.functions import monotonically_increasing_id

def make_predictions_with_ids(predictions, test_sdf_id):
    # row건수별로 0부터 순차적으로 증가하는 row_id 컬럼을 monotonically_increasing_id()을 이용하여 생성. 
    test_sdf_id = test_sdf_id.withColumn("row_id", monotonically_increasing_id())
    predictions = predictions.withColumn("row_id", monotonically_increasing_id())
    predictions = test_sdf_id.join(predictions, ("row_id")).drop("row_id")
    return predictions

predictions = make_predictions_with_ids(predictions, test_sdf_id)
display(predictions.limit(10))

In [0]:
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as F

# reordered가 1일 경우의 threshold를 0.5가 아니라 0.21로 설정하여 다시 reordered 예측 값 재 설정.  
def make_threshold_reordered(predictions, reorder_threshold):
    #vector를 array로 변환
    predictions = predictions.withColumn("probability_arr", vector_to_array('probability'))
    # 1일 때(즉 재주문)의 확률을 추출
    predictions = predictions.withColumn('1_proba', F.col('probability_arr')[1])
    # 1_proba값이 reorder_threshold보다 크면 1, 그렇지 않으면 0으로 reordered 컬럼 추가.
    predictions = predictions.withColumn('reordered', (F.col('1_proba') > reorder_threshold).cast('int'))
    
    return predictions

predictions = make_threshold_reordered(predictions, 0.21)

In [0]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType

# kaggle submission 파일 생성. 
def make_submission(predictions):
    predictions_grp = predictions.groupby('order_id').agg(F.count('*').alias('total_cnt_by_order_id'), 
                                                      F.sum(F.col('reordered')).alias('reordered_cnt'))
    
    # collect_list('product_id')로 입력되는 product_id list값을 ' '으로 결합된 문자열로 변환하는 함수 생성
    def get_product_ids_str(product_id_group):
        #product_id_group은 collect_list('product_id')로 group by된 집합으로 product_id를 list로 가지고 있는 형태로 입력 됨. 
        product_ids_str = ''
        for product_id in product_id_group:
            product_ids_str += ' ' + str(product_id)

        return product_ids_str

    # 일반 python용 UDF를 pyspark용 UDF로 변환. udf(lambda 입력변수: 일반 UDF, 해당 일반 UDF의 반환형)
    udf_get_product_ids_str = udf(lambda x:get_product_ids_str(x), StringType())
    
    # submission 생성
    submission_01 = predictions.filter('reordered == 1').groupBy('order_id').agg(udf_get_product_ids_str(F.collect_list('product_id')).alias('products'))
    submission_02 = predictions_grp.filter('reordered_cnt == 0').withColumn('products', F.lit('None')).select('order_id', 'products')
    submission = submission_01.union(submission_02)
    
    return submission

submission = make_submission(predictions)

In [0]:
display(submission.limit(76000))

### hyperopt로 LightGBM 하이퍼 파라미터 튜닝하기
* 학습 데이터를 다시 학습과 검증 데이터로 분리(isVal컬럼을 추가하여 True일 경우 검증, False일 경우 학습)
* hyperopt의 objective 함수 생성시 f1 score로 evaluation 하여 f1 score가 최소일 때의 hyper parameter 추출. 단 f1 score를 계산 시에 1의 prediction확률을 0.21 보정하여 계산.

In [0]:
%sql
drop table if exists train_data;

CREATE TABLE train_data
USING parquet
OPTIONS (
       path "/user/hive/warehouse/train_data/" );

drop table if exists test_data;

CREATE TABLE test_data
USING parquet
OPTIONS (
       path "/user/hive/warehouse/test_data/" );  

In [0]:
from pyspark.sql.functions import lit

# create table using parquet 로 로딩한 테이블에 대해서 delta format 확인 하지 말것. 
spark.sql("set spark.databricks.delta.formatCheck.enabled=false")
print(spark.catalog.listTables())

# train_data, test_data 테이블을 DataFrame으로 변환.
train_sdf = spark.table("train_data") # spark.sql("select * from train_data")
test_sdf = spark.table("test_data") # spark.sql("select * from test_data")
print('train_sdf type:', type(train_sdf), 'test_sdf type:', type(test_sdf))
print(train_sdf.printSchema())

# Null값은 모두 0으로 설정. 
train_sdf = train_sdf.fillna(0)

# validation 검증을 위한 isVal 컬럼 생성. 전체 학습 데이터의 20%를 isVal=True 설정. 
tr_sdf, val_sdf = train_sdf.randomSplit([0.8, 0.2], seed=2021)
tr_sdf = tr_sdf.withColumn('isVal', lit(False))
val_sdf = val_sdf.withColumn('isVal', lit(True))

tr_val_sdf = tr_sdf.union(val_sdf)
print("### count 확인:", train_sdf.count(), tr_sdf.count(), val_sdf.count(), tr_val_sdf.count())
display(tr_val_sdf)

In [0]:
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as F
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType

# submit 파일을 만들기 위한 order_id, product_id 용 test_sdf_id와 id가 제거된 feature 속성만을 가진 test_sdf 생성. 
def make_test_sdf_id_n_test_sdf(test_sdf):
    test_sdf_id = test_sdf.select('user_id', 'product_id', 'order_id')
    test_sdf = test_sdf.drop('user_id', 'product_id', 'order_id')
    return test_sdf_id, test_sdf

#예측 결과 predictions와 test_sdf_id를 조인하여 붙이기.
def make_predictions_with_ids(predictions, test_sdf_id):
    # row건수별로 0부터 순차적으로 증가하는 row_id 컬럼을 monotonically_increasing_id()을 이용하여 생성. 
    test_sdf_id = test_sdf_id.withColumn("row_id", monotonically_increasing_id())
    predictions = predictions.withColumn("row_id", monotonically_increasing_id())
    predictions = test_sdf_id.join(predictions, ("row_id")).drop("row_id")
    return predictions

# reordered가 1일 경우의 threshold를 0.5가 아니라 0.21로 설정하여 다시 reordered 예측 값 재 설정.  
def make_threshold_reordered(predictions, reorder_threshold):
    #vector를 array로 변환
    predictions = predictions.withColumn("probability_arr", vector_to_array('probability'))
    # 1일 때(즉 재주문)의 확률을 추출
    predictions = predictions.withColumn('1_proba', F.col('probability_arr')[1])
    # 1_proba값이 reorder_threshold보다 크면 1, 그렇지 않으면 0으로 reordered 컬럼 추가.
    predictions = predictions.withColumn('reordered', (F.col('1_proba') > reorder_threshold).cast('int'))
    
    return predictions

# reordered가 1일 경우의 threshold를 0.5가 아니라 0.21로 설정하여 다시 reordered 예측 값 재 설정.  
# f1 score 평가를 위해서 threshold를 0.21 기준으로 변경하여 다시 prediction 값을 update. predictions을 double 형으로 변경. 
def make_threshold_reordered_pred(predictions, reorder_threshold):
    #vector를 array로 변환
    predictions = predictions.withColumn("probability_arr", vector_to_array('probability'))
    # 1일 때(즉 재주문)의 확률을 추출
    predictions = predictions.withColumn('1_proba', F.col('probability_arr')[1])
    # 1_proba값이 reorder_threshold보다 크면 1, 그렇지 않으면 0으로 prediction 컬럼 update.
    predictions = predictions.withColumn('prediction', (F.col('1_proba') > reorder_threshold).cast('double')) # prediction은 update하되 double로 . 
    
    return predictions

# kaggle submission 파일 생성. 
def make_submission(predictions):
    predictions_grp = predictions.groupby('order_id').agg(F.count('*').alias('total_cnt_by_order_id'), 
                                                      F.sum(F.col('reordered')).alias('reordered_cnt'))
    
    # collect_list('product_id')로 입력되는 product_id list값을 ' '으로 결합된 문자열로 변환하는 함수 생성
    def get_product_ids_str(product_id_group):
        #product_id_group은 collect_list('product_id')로 group by된 집합으로 product_id를 list로 가지고 있는 형태로 입력 됨. 
        product_ids_str = ''
        for product_id in product_id_group:
            product_ids_str += ' ' + str(product_id)

        return product_ids_str

    # 일반 python용 UDF를 pyspark용 UDF로 변환. udf(lambda 입력변수: 일반 UDF, 해당 일반 UDF의 반환형)
    udf_get_product_ids_str = udf(lambda x:get_product_ids_str(x), StringType())
    
    # submission 생성
    submission_01 = predictions.filter('reordered == 1').groupBy('order_id').agg(udf_get_product_ids_str(F.collect_list('product_id')).alias('products'))
    submission_02 = predictions_grp.filter('reordered_cnt == 0').withColumn('products', F.lit('None')).select('order_id', 'products')
    submission = submission_01.union(submission_02)
    
    return submission

In [0]:
from pyspark.ml.feature import VectorAssembler
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from mmlspark.lightgbm import LightGBMClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

search_space = {'maxDepth': hp.quniform('maxDepth', 10, 20, 1),
                'numLeaves': hp.quniform('numLeaves', 32, 78, 4),
                'baggingFraction': hp.uniform('baggingFraction', 0.6, 1.0),
                'featureFraction': hp.uniform('featureFraction', 0.6, 1.0),
                'learningRate': hp.uniform('learningRate', 0.01, 0.1)
               }

# feature vectorization 적용할 column명 추출. label 컬럼인 reordered는 제외
vector_columns = [column_name for column_name, column_type in train_sdf.dtypes if column_name != 'reordered']

vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
tr_sdf_vectorized = vector_assembler.transform(tr_sdf)
val_sdf_vectorized = vector_assembler.transform(val_sdf)

f1_eval = MulticlassClassificationEvaluator(labelCol='reordered', predictionCol='prediction', metricName='f1' )

def objective_func(space):
    lgbm_classifier = LightGBMClassifier(numIterations=50, maxDepth=int(space['maxDepth'])
                                        , numLeaves=int(space['numLeaves'])
                                        , baggingFraction=space['baggingFraction']
                                        , featureFraction=space['featureFraction']
                                        , learningRate=space['learningRate']
                                        , featuresCol='features', labelCol='reordered')
    
    lgbm_model = lgbm_classifier.fit(tr_sdf_vectorized)
    predictions = lgbm_model.transform(val_sdf_vectorized)
    # predictions의 prediction 컬럼을 probability가 0.21보다 크면 1로 변경.
    predictions = make_threshold_reordered_pred(predictions, 0.21)
    f1_eval = MulticlassClassificationEvaluator(labelCol='reordered', predictionCol='prediction', metricName='f1' )
    f1_score = f1_eval.evaluate(predictions)
    
    print('space:', space, "f1_score:", f1_score)
    
    return {'loss': -1*f1_score, 'status':STATUS_OK }


algo = tpe.suggest
trials = Trials()

best = fmin(fn=objective_func, space=search_space, max_evals=25, algo=algo, trials=trials)

In [0]:
print('best:', best)

#### databricks cluster의 자원 부족 현상으로 위의 hyperparameter 코드가 제대로 수행되지 않을 경우 아래로 대체. 
* {'baggingFraction': 0.741521593665619, 'featureFraction': 0.9432850679715429, 'learningRate': 0.013659016471159318, 'maxDepth': 18.0, 'numLeaves': 72.0}
* {'baggingFraction': 0.816383152884725, 'featureFraction': 0.9338421018465028, 'learningRate': 0.010062170204073871, 'maxDepth': 13.0, 'numLeaves': 44.0}

In [0]:
from pyspark.ml.feature import VectorAssembler
from mmlspark.lightgbm import LightGBMClassifier

vector_columns = [column_name for column_name, column_type in train_sdf.dtypes if column_name != 'reordered']
vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
train_sdf_vectorized = vector_assembler.transform(train_sdf)
lgbm_estimator = LightGBMClassifier(featuresCol="features", labelCol="reordered"
                                    , numLeaves=72, maxDepth=18, numIterations=100
                                    , baggingFraction=0.741, featureFraction=0.943, learningRate=0.013
                                   )

lgbm_model = lgbm_estimator.fit(train_sdf_vectorized)

In [0]:
# test_sdf에서 id를 제거한 데이터와, id만 가지는 데이터를 분리
test_sdf_id, test_sdf = make_test_sdf_id_n_test_sdf(test_sdf)
test_sdf = test_sdf.fillna(0)

test_sdf_vectorized = vector_assembler.transform(test_sdf)
predictions = lgbm_model.transform(test_sdf_vectorized)

In [0]:
predictions = make_predictions_with_ids(predictions, test_sdf_id)
display(predictions.limit(10))

In [0]:
predictions = make_threshold_reordered(predictions, 0.21)
display(predictions.limit(10))

In [0]:
submission = make_submission(predictions)

In [0]:
display(submission.limit(76000))